##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Индивидуальное обучение: пошаговое руководство

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/customization/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/customization/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/customization/custom_training_walkthrough.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом руководстве используется машинное обучение для *классификации* цветов ириса по видам. Мы используем TensorFlow для:
1. Построения модели.
2. Обучения этой модели на примерах данных.
3. Использование модели, для создания прогноза на неизвестных данных.

## Программирование c TensorFlow

В этом руководстве используются следующие общие концепции TensorFlow:

* Используйте TensorFlow [eager execution](../../guide/eager.ipynb) ,
* Импортируйте данные с помощью [Datasets API](../../guide/datasets.ipynb),
* Создавайте модели и слои с помощью [Keras API](../../guide/keras/overview.ipynb).

Это руководство построено так же, как и многие программы TensorFlow:

1. Импортируйте и проанализируйте набор данных.
2. Выберите тип модели.
3. Обучите модель.
4. Оцените эффективность модели.
5. Используйте обученную модель, чтобы делать прогнозы.

## Установка

### Настройка импорта

Импортируйте TensorFlow и другие необходимые модули Python. По умолчанию TensorFlow использует [активное выполнение](../../guide/eager.ipynb) для немедленного вычисления операций, возвращая конкретные значения вместо создания вычислительного графа, который выполняется позже. Если вы привыкли к REPL или интерактивной консоли python, это примерно тоже.

In [ ]:
import os
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

## Проблема классификации цветов ириса

Представьте, что вы ботаник, ищущий автоматизированный способ классифицировать каждый найденный цветок ириса. Машинное обучение предоставляет множество алгоритмов для статистической классификации цветов. Например, сложная программа машинного обучения может классифицировать цветы на основе фотографий. Наши амбиции более скромные - мы собираемся классифицировать цветы ириса на основе размеров их [чашелистиков](https://en.wikipedia.org/wiki/Sepal) и [лепестков](https://en.wikipedia.org/wiki/Petal).

Род Iris включает около 300 видов, но наша программа классифицирует только следующие три:

* Iris setosa
* Iris virginica
* Iris versicolor

<table>
  <tr><td>
    <img src="https://www.tensorflow.org/images/iris_three_species.jpg"
         alt="Petal geometry compared for three iris species: Iris setosa, Iris virginica, and Iris versicolor">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://commons.wikimedia.org/w/index.php?curid=170298">Iris setosa</a> (by <a href="https://commons.wikimedia.org/wiki/User:Radomil">Radomil</a>, CC BY-SA 3.0), <a href="https://commons.wikimedia.org/w/index.php?curid=248095">Iris versicolor</a>, (by <a href="https://commons.wikimedia.org/wiki/User:Dlanglois">Dlanglois</a>, CC BY-SA 3.0), and <a href="https://www.flickr.com/photos/33397993@N05/3352169862">Iris virginica</a> (by <a href="https://www.flickr.com/photos/33397993@N05">Frank Mayfield</a>, CC BY-SA 2.0).<br/>&nbsp;
  </td></tr>
</table>

К счастью, кто-то уже создал [набор данных из 120 цветов ириса](https://en.wikipedia.org/wiki/Iris_flower_data_set) с размерами чашелистиков и лепестков. Это классический набор данных, популярный при решении задач классификации машинного обучения для начинающих.

## Импорт и анализ тренировочного датасета

Загрузите файл и преобразуйте его в структуру, которая может использоваться программой Python.

### Загрузка датасета

Загрузите файл тренировочного датасета с помощью функции tf.keras.utils.get_file. Этот метод возвращает путь к загруженному файлу:

In [ ]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url)

print("Локальная копия датасета: {}".format(train_dataset_fp))

### Проверка данных

Этот датасет, `iris_training.csv`, представляет собой простой текстовый файл, в котором хранятся табличные данные, отформатированные как значения, разделенные запятыми (CSV). Используйте команду `head -n5`, чтобы взглянуть на первые пять записей:

In [ ]:
!head -n5 {train_dataset_fp}

Обратите внимание на следующее:

1. Первая строка - это заголовок, содержащий информацию о датасете:
   * Всего 120 примеров. В каждом примере есть четыре признака и один из трех возможных ярлыков.
2. Последующие строки представляют собой записи данных, по одной *[пример](https://developers.google.com/machine-learning/glossary/#example)* на строку, где:
   * Первые четыре поля - это *[признаки](https://developers.google.com/machine-learning/glossary/#feature)* примера. Здесь поля содержат числа с плавающей запятой, представляющие размеры цветов.
   * Последний столбец - это *[ярлык](https://developers.google.com/machine-learning/glossary/#label)*: это значение, которое мы хотим предсказать. Для этого набора данных это целое число 0, 1 или 2, соответствующее названию цветка.

Запишем это в коде:

In [ ]:
# столбцы в CSV файле
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

feature_names = column_names[:-1]
label_name = column_names[-1]

print("Признаки: {}".format(feature_names))
print("Ярлык: {}".format(label_name))

Каждый ярлык связана с именем строки(например, «сетоса»), но машинное обучение обычно основывается на числовых значениях. Номера ярлыков соответствуют именам, например:

* `0`: Ирис сетоса
* `1`: Ирис разноцветный
* `2`: Ирис виргинский

Дополнительные сведения о функциях и ярлыках см. В разделе [Терминология машинного обучения](https://developers.google.com/machine-learning/crash-course/framing/ml-terminology).

In [ ]:
class_names = ['Iris setosa', 'Iris versicolor', 'Iris virginica']

### Создание объекта `tf.data.Dataset`

TensorFlow [Dataset API](../../guide/data.ipynb) обрабатывает наиболее распространенные случаи загрузки данных в модель. Это API высокого уровня для чтения данных и преобразования их в форму, используемую для обучения.

Поскольку набор данных представляет собой текстовый файл в формате CSV, используйте функцию `tf.data.experimental.make_csv_dataset`, чтобы прочитать csv файл и преобразовать данные в нужный формат. Так как эта функция генерирует данные для моделей, которые используются для изучения ML, ее поведение по умолчанию - перемешать данные (`shuffle = True, shuffle_buffer_size = 10000`) и бесконечно повторять пакеты(` num_epochs = None`). Мы также устанавливаем размер пакета [batch_size](https://developers.google.com/machine-learning/glossary/#batch_size):

In [ ]:
batch_size = 32

train_dataset = tf.data.experimental.make_csv_dataset(
    train_dataset_fp,
    batch_size,
    column_names=column_names,
    label_name=label_name,
    num_epochs=1)

Функция `make_csv_dataset` возвращает объект `tf.data.Dataset`, содержащий пары `(features, label)` , где `features` - словарь: `{'feature_name': value}`

Объекты `Dataset` итерируемые. Давайте посмотрим на набор признаков:

In [ ]:
features, labels = next(iter(train_dataset))

print(features)

Обратите внимание, что одинаковые признаки сгруппированы вместе или *пакетированы*. Поля каждой строки добавляются к соответствующему массиву признаков. Измените batch_size, чтобы установить другое количество признаков, хранящихся в этих массивах, ранее мы установили его равным 32.

Вы можете увидеть несколько кластеры, построив график признаков из пакета:

In [ ]:
plt.scatter(features['petal_length'],
            features['sepal_length'],
            c=labels,
            cmap='viridis')

plt.xlabel("Длина лепестка")
plt.ylabel("Длина чашелистика")
plt.show()

Чтобы упростить процесс построения модели, создайте функцию для перепаковки словаря признаков в единый массив с формой: `(batch_size, num_features)`.

Эта функция использует метод tf.stack, который принимает значения из списка тензоров и создает комбинированный тензор указанной размерности:

In [ ]:
def pack_features_vector(features, labels):
  """Упаковка признаков и одномерный массив."""
  features = tf.stack(list(features.values()), axis=1)
  return features, labels

Затем используйте метод `tf.data.Dataset.map`, чтобы упаковать признаки каждой пары `(features, label)` в обучающий датасет:

In [ ]:
train_dataset = train_dataset.map(pack_features_vector)

Теперь каждый элемент features в `Dataset` представляет собой массив с размерностью `(batch_size, num_features)`. Давайте посмотрим на первые тензоры:

In [ ]:
features, labels = next(iter(train_dataset))

print(features[:5])
print(labels[:5])

## Выбор типа модели

### Почему именно модель?

*[Модель](https://developers.google.com/machine-learning/crash-course/glossary#model)* - это связь между признаками и меткой. Для задачи классификации цветов ириса модель определяет взаимосвязь между размерами чашелистиков и лепестков и прогнозируемыми видами ирисов. Некоторые простые модели можно описать с помощью нескольких строк алгебры, но сложные модели машинного обучения имеют большое количество параметров, которые сложно обобщать.

Смогли бы вы определить взаимосвязь между четырьмя характеристиками цветка и видом ириса *без* машинного обучения? То есть, можете ли вы использовать традиционные методы программирования (например, множество условных операторов) для создания модели? Возможно, если достаточно долго анализировать набор данных, у вас получится определить взаимосвязь между размерами лепестков и чашелистиков для конкретного вида. Но это становится трудным, или даже невозможным для более сложных наборов данных. Хороший подход к машинному обучению *модель определяет за вас*. Если вы загрузите достаточное количество репрезентативных примеров в правильный тип модели машинного обучения, программа сама определит взаимосвязи.

### Выбор модели

Нам нужно выбрать модель для обучения. Есть множество типов моделей, и чтобы выбрать хорошую, нужен опыт. В этом руководстве для решения проблемы классификации цветков ириса используется нейронная сеть. *[Нейронные сети](https://developers.google.com/machine-learning/glossary/#neural_network)* могут обнаруживать сложные отношения между признаками и меткой. Это хорошо структурированный граф, организованный в один или несколько *[скрытых слоев](https://developers.google.com/machine-learning/glossary/#hidden_layer)*. Каждый скрытый слой состоит из одного или нескольких *[нейронов](https://developers.google.com/machine-learning/glossary/#neuron)*. Существует несколько категорий нейронных сетей, и эта программа использует плотную или *[полносвязанную нейронную сеть](https://developers.google.com/machine-learning/glossary/#fully_connected_layer)*: нейроны следующего слоя получают входные параметры от *каждого* нейрона предыдущего слоя. Например, на рис.2 показана полносвязная нейронная сеть, состоящая из входного слоя, двух скрытых слоев и выходного слоя:

<table>
  <tr><td>
    <img src="https://www.tensorflow.org/images/custom_estimators/full_network.png"
         alt="A diagram of the network architecture: Inputs, 2 hidden layers, and outputs">
  </td></tr>
  <tr><td align="center">
    <b>Figure 2.</b> A neural network with features, hidden layers, and predictions.<br/>&nbsp;
  </td></tr>
</table>

Когда модель, изображенная на рис.2, обучается и ей подается немаркированный пример, она дает три прогноза: вероятность того, что этот цветок принадлежит данному виду ириса. Этот прогноз называется *[вывод(inference)](https://developers.google.com/machine-learning/crash-course/glossary#inference)*. В этом примере сумма прогнозов на выходе равна 1.0. На рис.2 этот прогноз разделен на: 0,02 для *Iris setosa*, 0,95 для *Iris versicolor* и 0,03 для *Iris virginica*. Это означает, что модель предсказывает с вероятностью 95%, что пример цветка без метки - это *Iris versicolor*.

### Создание модели с использованиемng Keras

API TensorFlow `tf.keras` - предпочтительный способ создания моделей и слоев. Этот способ позволяет легко создавать модели и экспериментировать с ними, в то время как `Keras` справляется со сложностью соединения всего вместе.

Модель `tf.keras.Sequential` представляет собой последовательный набор слоев. Его конструктор принимает список экземпляров слоя, в данном случае два слоя `tf.keras.layers.Dense` с 10 узлами в каждом и выходной слой `tf.keras.layers.Dense` с 3 узлами, представляющими наши предсказания(так как у нас всего 3 вида ириса). Параметр `input_shape` первого слоя соответствует количеству объектов из набора данных и является обязательным:

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(4,)),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(3)
])

*[Функция активации](https://developers.google.com/machine-learning/crash-course/glossary#activation_function)* определяет форму вывода каждого узла в слое. Эти функции важны - без них модель была бы эквивалентна одному слою. Существует много `tf.keras.activations`, но [ReLU](https://developers.google.com/machine-learning/crash-course/glossary#ReLU) является обычным для скрытых слоев.

Идеальное количество скрытых слоев и нейронов зависит от проблемы и набора данных. Как и многие другие аспекты машинного обучения, выбор наилучшей формы нейронной сети требует сочетания знаний и экспериментов. Как показывает практика, увеличение количества скрытых слоев и нейронов обычно создает более мощную модель, которая требует **большего количества данных** для эффективного обучения.

### Использование модели

Давайте быстро посмотрим, что эта модель делает с рядом признаков:

In [ ]:
predictions = model(features)
predictions[:5]

Здесь каждый пример возвращает [logit](https://developers.google.com/machine-learning/crash-course/glossary#logits) для каждого класса.

Чтобы преобразовать эти немасштабированные значения в вероятность для каждого класса, используйте функцию [softmax](https://developers.google.com/machine-learning/crash-course/glossary#softmax):

In [ ]:
tf.nn.softmax(predictions[:5])

Взяв `tf.argmax` из полученных классов, мы получаем прогнозируемый индекс класса. Но модель еще не была обучена, поэтому прогнозы по большей части неправильные:

In [ ]:
print("Прогноз: {}".format(tf.argmax(predictions, axis=1)))
print("Реальный класс: {}".format(labels))

## Обучение модели

*[Обучение](https://developers.google.com/machine-learning/crash-course/glossary#training)* - это этап машинного обучения, на котором модель постепенно оптимизируется, или, другими словами, модель *изучает* набор данных. Цель обучения состоит в том, чтобы узнать достаточно о структуре обучающих данных и делать прогнозы относительно невидимых данных. Если вы узнаете *слишком много* об обучающем наборе данных, то прогнозы будут работать только для тех данных, которые модель уже видела, и не будут обобщать основные признаки. Эта проблема называется *[переобучение](https://developers.google.com/machine-learning/crash-course/glossary#overfitting)* - это как запоминание ответов, а не понимание того, как решить проблему.

Проблема классификации Iris - это пример *[машинного обучения с учителем](https://developers.google.com/machine-learning/glossary/#supervised_machine_learning)*: модель обучается на примерах, содержащих метки. 
В *[машинном обучение без учителя](https://developers.google.com/machine-learning/glossary/#unsupervised_machine_learning)* примеры не содержат ярлыков. Вместо этого модель пытается находить закономерности среди функций.

### Определение функции потерь и градиента

На этапах обучения и оценки необходимо рассчитать *[потери модели](https://developers.google.com/machine-learning/crash-course/glossary#loss)*. Функции потерь измеряют, насколько прогнозы модели отличаются от желаемой метки, другими словами, насколько плохо модель работает. Мы хотим минимизировать или оптимизировать это значение.

Наша модель будет рассчитывать потери с помощью функции `tf.keras.losses.SparseCategoricalCrossentropy`, которая принимает прогнозируемые вероятности класса и реальную метку и возвращает средние потери.

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
def loss(model, x, y, training):
  # training = training нужно только если есть слои с разными
  # поведением во время обучения по сравнению с выводом (например, слой Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)


l = loss(model, features, labels, training=False)
print("Величина потерь: {}".format(l))

Используйте `tf.GradientTape` для вычисления *[градиентов](https://developers.google.com/machine-learning/crash-course/glossary#gradient)*, используемых для оптимизации вашей модели:

In [ ]:
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

### Создание оптимайзера

*[Оптимайзер](https://developers.google.com/machine-learning/crash-course/glossary#optimizer)* применяет вычисленные градиенты к переменным модели, чтобы минимизировать `функцию потерь`. Вы можете представить функцию потерь как изогнутую поверхность (см. Рисунок 3), и мы хотим найти ее самую низкую точку. Градиенты указывают на направление наискорейшего подъема, поэтому мы пойдем в обратном направлении и спустимся вниз. Итеративно вычисляя потери и градиент для каждого пакета данных, мы скорректируем модель во время обучения. Постепенно модель найдет наилучшее сочетание весов и смещения, чтобы минимизировать потери. И чем меньше потери, тем лучше прогнозы модели.

<table>
  <tr><td>
    <img src="https://cs231n.github.io/assets/nn3/opt1.gif" width="70%"
         alt="Optimization algorithms visualized over time in 3D space.">
  </td></tr>
  <tr><td align="center">
    <b>Figure 3.</b> Optimization algorithms visualized over time in 3D space.<br/>(Source: <a href="http://cs231n.github.io/neural-networks-3/">Stanford class CS231n</a>, MIT License, Image credit: <a href="https://twitter.com/alecrad">Alec Radford</a>)
  </td></tr>
</table>

В TensorFlow есть множество алгоритмов оптимизации, доступных для обучения. Наша модель использует `tf.keras.optimizers.SGD`, который реализует алгоритм *[стохастический градиентный спуск](https://developers.google.com/machine-learning/crash-course/glossary#gradient_descent)* (SGD). `learning_rate` устанавливает размер шага для каждой итерации спуска. Это *гиперпараметр*, который вы обычно настраиваете для достижения лучших результатов.

Давайте определим оптимайзер:

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

Используем его для расчета одного шага оптимизации:

In [ ]:
loss_value, grads = grad(model, features, labels)

print("Шаг: {}, Начальная величина потерь: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Шаг: {}, Величина потерь: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))

### Тренировочный цикл

Когда все детали на месте, модель готова к обучению! Цикл обучения передает в модель пакеты тренировочных данных, чтобы помочь ей делать более точные прогнозы. Следующий блок кода определяет шаги обучения:

1. Итерировать каждую *эпоху*. Эпоха - это один проход по набору данных.
2. В течение эпохи перебирать каждый пример в обучающем `Dataset`, захватывая его *признаки* (` x`) и *метку* (`y`).
3. Используя признаки набора данных, сделать прогноз и сравнить его с меткой. Измерить ошибочность прогноза и использовать ее для расчета величины потерь и градиентов модели.
4. Использовать `optimizer`, чтобы обновить переменные модели.
5. Следить за статистикой для визуализации.
6. Повторить предыдущие шаги для каждой эпохи.

Переменная `num_epochs` - это количество повторений цикла прохода модели по всему датасету. Как ни странно, более длительное обучение не гарантирует получение лучшей модели. `num_epochs` - это *[гиперпараметр](https://developers.google.com/machine-learning/glossary/#hyperparameter)*, который вы можете настраивать. Выбор правильного числа обычно требует как опыта, так и экспериментов:

In [ ]:
## Примечание. При повторном запуске этой ячейки используются те же переменные модели.

# Сохраним результаты для графиков
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Тренировочный цикл - используем пакеты по 32 записи
  for x, y in train_dataset:
    # Оптимизируем
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Отслеживаем прогресс
    epoch_loss_avg.update_state(loss_value)  # Добавляем величину потерь текущего пакета
    # Сравниваем спрогнозированную и реальную метки
    # training=True нужно только если есть слои с разными
    # поведением во время обучения по сравнению с выводом (например, слой Dropout).
    epoch_accuracy.update_state(y, model(x, training=True))

  # Конец эпохи
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Эпоха {:03d}: Потери: {:.3f}, Точность: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

### Визуализация функции потерь

Хотя полезно выводить прогресс обучения модели, часто бывает полезнее увидеть графическое представление этого прогресса. [TensorBoard](https://www.tensorflow.org/tensorboard) - хороший инструмент визуализации, который входит в состав TensorFlow, но мы можем создавать базовые диаграммы с помощью модуля matplotlib.

Интерпретация этих графиков требует некоторого опыта, но вы должны стремится к тому, чтобы потери уменьшались, а точность повышалась:

In [ ]:
fig, axes = plt.subplots(2, sharex=True, figsize=(12, 8))
fig.suptitle('Training Metrics')

axes[0].set_ylabel("Loss", fontsize=14)
axes[0].plot(train_loss_results)

axes[1].set_ylabel("Accuracy", fontsize=14)
axes[1].set_xlabel("Epoch", fontsize=14)
axes[1].plot(train_accuracy_results)
plt.show()

## Оценка эффективности модели

Теперь, когда модель обучена, мы можем получить некоторую статистику о ее производительности.

*Оценка модели* означает определение того, насколько эффективно модель делает прогнозы. Чтобы определить эффективность модели при классификации цветков ириса, передайте модели некоторые величины чашелистиков и лепестков и попросите модель предсказать, какие виды ирисов они представляют. Затем сравните прогнозы модели с фактическими данными. Например, модель, которая выбрала правильный вид в половине примеров входных данных, имеет *[точность](https://developers.google.com/machine-learning/glossary/#accuracy)* `0.5`. На рисунке 4 показана более эффективная модель, в которой 4 из 5 прогнозов верны и точность составляет 80%:

<table cellpadding="8" border="0">
  <colgroup>
    <col span="4" >
    <col span="1" bgcolor="lightblue">
    <col span="1" bgcolor="lightgreen">
  </colgroup>
  <tr bgcolor="lightgray">
    <th colspan="4">Example features</th>
    <th colspan="1">Label</th>
    <th colspan="1" >Model prediction</th>
  </tr>
  <tr>
    <td>5.9</td><td>3.0</td><td>4.3</td><td>1.5</td><td align="center">1</td><td align="center">1</td>
  </tr>
  <tr>
    <td>6.9</td><td>3.1</td><td>5.4</td><td>2.1</td><td align="center">2</td><td align="center">2</td>
  </tr>
  <tr>
    <td>5.1</td><td>3.3</td><td>1.7</td><td>0.5</td><td align="center">0</td><td align="center">0</td>
  </tr>
  <tr>
    <td>6.0</td> <td>3.4</td> <td>4.5</td> <td>1.6</td> <td align="center">1</td><td align="center" bgcolor="red">2</td>
  </tr>
  <tr>
    <td>5.5</td><td>2.5</td><td>4.0</td><td>1.3</td><td align="center">1</td><td align="center">1</td>
  </tr>
  <tr><td align="center" colspan="6">
    <b>Figure 4.</b> An Iris classifier that is 80% accurate.<br/>&nbsp;
  </td></tr>
</table>

### Тестовый датасет

Оценка модели аналогична обучению модели. Основная разница в том, что примеры взяты из отдельного *[тестового датасета](https://developers.google.com/machine-learning/crash-course/glossary#test_set)*, а не из датасета для обучения. Чтобы достоверно оценить эффективность модели, примеры, используемые для оценки модели, должны отличаться от примеров, используемых для обучения модели.

Настройка тестового `Dataset` аналогична настройке тренировочного `Dataset`. Загрузите текстовый файл CSV и прочитайте из него данные, а затем немного перемешайте:

In [ ]:
test_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"

test_fp = tf.keras.utils.get_file(fname=os.path.basename(test_url),
                                  origin=test_url)

In [ ]:
test_dataset = tf.data.experimental.make_csv_dataset(
    test_fp,
    batch_size,
    column_names=column_names,
    label_name='species',
    num_epochs=1,
    shuffle=False)

test_dataset = test_dataset.map(pack_features_vector)

### Оценка модели на тестовом датасете

В отличие от этапа обучения, модель оценивается только одной [эпохой](https://developers.google.com/machine-learning/glossary/#epoch) тестовых данных. В следующих строках кода мы перебираем каждый пример в тестовом датасете и сравниваем прогноз модели с фактической меткой. Этот метод используется для измерения точности модели по всему набору тестов:

In [ ]:
test_accuracy = tf.keras.metrics.Accuracy()

for (x, y) in test_dataset:
  logits = model(x, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracy(prediction, y)

print("Точность на тестовых данных: {:.3%}".format(test_accuracy.result()))

Мы можем просмотреть последний пакет и убедится, что в большем количесвте случаев модель прогнозирует правильно:

In [ ]:
tf.stack([y,prediction],axis=1)

## Использование обученной модели

Мы обучили модель и «доказали», что она хороша(но не идеальна) в классификации видов ирисов. Теперь давайте воспользуемся обученной моделью, чтобы сделать некоторые прогнозы для [немаркированных примеров](https://developers.google.com/machine-learning/glossary/#unlabeled_example); то есть примеры, которые содержат признаки, но не содержат метку.

В реальной жизни призанки без меток могут быть получены из множества различных источников, включая приложения, файлы CSV или каналы данных. А пока мы попробуем загрузить в модель три немаркированных примера, чтобы предсказать их метки. Напомним, номера меток отображаются в именованное представление как:

* `0`: Iris setosa
* `1`: Iris versicolor
* `2`: Iris virginica

In [ ]:
predict_dataset = tf.convert_to_tensor([
    [5.1, 3.3, 1.7, 0.5,],
    [5.9, 3.0, 4.2, 1.5,],
    [6.9, 3.1, 5.4, 2.1]
])

# training=False нужно только если есть слои с разными
# поведением во время обучения по сравнению с выводом (например, слой Dropout).
predictions = model(predict_dataset, training=False)

for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))